In [1]:
fundos = {
    "C0000463698" : "Trigono Delphos",         
    "C0000570494" : "BB TD Pre",
    "C0000570516" : "BB TD Infl",
    "C0000570524" : "BB TD Infl Curta",
    "C0000496121" : "BB FX Bonds",
    "C0000207098" : "BB TD Selic",
    "C0000457371" : "BB High",
    "C0000420530" : "Artesanal RF",
    "C0000377856" : "Mag Cash RF",
    "C0000422991" : "Ouro Preto Real RF",
    "C0000337031" : "Western SP500 FIM",
    "C0000679909" : "V8 L&S"
}


In [2]:
import time
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.covariance import LedoitWolf
from scipy.cluster.hierarchy import linkage, leaves_list
from scipy.spatial.distance import squareform


# =========================================
# Selenium Setup
# =========================================
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--disable-gpu")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

def to_float(series):

    s = series.astype(str)

    if s.str.contains(",").any():
        # padrão BR
        s = s.str.replace(".", "", regex=False)
        s = s.str.replace(",", ".", regex=False)

    s = s.str.replace("R$", "", regex=False)
    s = s.str.strip()

    return pd.to_numeric(s, errors="coerce")


# =========================================
#  Função scraping tabela
# =========================================
def baixar_fundo(codigo):

    url = f"https://data.anbima.com.br/fundos/{codigo}/dados-periodicos"
    print("Scraping", codigo)

    driver.get(url)
    time.sleep(6)

    try:
        table = driver.find_element(By.TAG_NAME, "table")
        rows = table.find_elements(By.TAG_NAME, "tr")
    except:
        print("Tabela não encontrada:", codigo)
        return None

    dados = []

    for r in rows[1:]:
        cols = r.find_elements(By.TAG_NAME, "td")

        if len(cols) >= 3:

            data = cols[0].text.strip()
            cota = cols[2].text.strip()

            dados.append([data, cota])

    if len(dados) == 0:
        return None

    df = pd.DataFrame(dados, columns=["data", fundos[codigo]])

    df["data"] = pd.to_datetime(df["data"], dayfirst=True)

    df[fundos[codigo]] = to_float(df[fundos[codigo]])
  
    
    return df.set_index("data")


In [3]:
# =========================================
#  Baixar todos fundos
# =========================================
dfs = []

for cod in fundos.keys():
    d = baixar_fundo(cod)
    if d is not None:
        dfs.append(d)

driver.quit()

# merge
df_cotas = pd.concat(dfs, axis=1)#.sort_index()

print("\nCotas:")
print(df_cotas.tail())



Scraping C0000463698
Scraping C0000570494
Scraping C0000570516
Scraping C0000570524
Scraping C0000496121
Scraping C0000207098
Scraping C0000457371
Scraping C0000420530
Scraping C0000377856
Scraping C0000422991
Scraping C0000337031
Scraping C0000679909

Cotas:
            Trigono Delphos  BB TD Pre  BB TD Infl  BB TD Infl Curta  \
data                                                                   
2026-02-10         2.737802   1.501271    1.332928          1.521363   
2026-02-11         2.769356   1.502110    1.334639          1.522660   
2026-02-12         2.677559   1.503693    1.337347          1.524281   
2026-02-13         2.647082   1.505193    1.340781          1.526036   
2026-02-18              NaN        NaN         NaN               NaN   

            BB FX Bonds  BB TD Selic   BB High  Artesanal RF  Mag Cash RF  \
data                                                                        
2026-02-10     1.554582    10.377988  1.960152      2.407918     3.035833   
2026

In [ ]:
# ========================================
#  Retornos
# =========================================
retornos = (
    df_cotas
    .pct_change(fill_method=None)
    .dropna(how="any")
)

# retornos

: 

In [ ]:
# =========================================
#  Covariância ROBUSTA
# A covariância robusta de Ledoit-Wolf é um 
# método estatístico que melhora a estimativa 
# da matriz de covariância, especialmente quando 
# você tem:
# Poucos dados em relação ao número de ativos
# Ruído nos dados
# Instabilidade na matriz de covariância amostral
# =========================================
lw = LedoitWolf().fit(retornos.values)

cov_robusta = pd.DataFrame(
    lw.covariance_,
    index=retornos.columns,
    columns=retornos.columns
)

# Calcular a matriz de correlação a partir da covariância
std_dev = np.sqrt(np.diag(cov_robusta))
corr_robusta = cov_robusta / np.outer(std_dev, std_dev)
#  Garantir que a diagonal seja exatamente 1.0
np.fill_diagonal(corr_robusta.values, 1.0)


: 

In [ ]:
# Reordenamento hierárquico 
# Converte correlação em distância
dist_robusta = 1 - corr_robusta
# Clustering hierárquico. squareform() converte a matriz de distância para vetor condensado. linkage() agrupa os ativos usando método "average" (UPGMA)
link_robusta = linkage(squareform(dist_robusta), method="average")
# Extrai a ordem ótima das folhas do dendrograma
ordem_robusta = leaves_list(link_robusta)
# Reordena a matriz de correlação
corr_robusta_ord = corr_robusta.iloc[ordem_robusta, ordem_robusta]


: 

In [ ]:
# Criar heatmap
plt.figure(figsize=(10, 7))
sns.heatmap(
    corr_robusta_ord,
    annot=True,
    fmt='.2f',
    cmap='coolwarm',
    center=0,
    vmin=-1,
    vmax=1,
    # square=True,
    cbar_kws={'label': 'Correlação'}
)
plt.title('Matriz de Correlação Robusta Reordenada (Ledoit-Wolf)')
plt.tight_layout()
plt.show()

: 

In [ ]:
# Correlação
corr = retornos.corr()
# Reordenamento hierárquico 
dist = 1 - corr
link = linkage(squareform(dist), method="average")
ordem = leaves_list(link)
corr_ord = corr.iloc[ordem, ordem]


: 

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(
    corr_ord,
    annot=True,
    fmt='.2f',
    cmap='coolwarm',
    center=0,
    vmin=-1,
    vmax=1,
    cbar_kws={'label': 'Correlação'}
)
plt.title('Matriz de Correlação')
plt.tight_layout()
plt.show()

: 

In [ ]:
from  DT_atualiza_settings import gc

: 

In [ ]:
planilha = gc.open('Investimentos')
pagina = planilha.worksheet("Cotas-Fundos")

: 

In [ ]:
# mask = df_cotas.isin([np.inf, -np.inf]) | df_cotas.isna()
# problematic_rows = df_cotas[mask.any(axis=1)]
# print("\nProblematic rows:")
# print(problematic_rows)

: 

In [ ]:
df_cotas.insert(0, "data", df_cotas.index)
df_cotas['data']= df_cotas['data'].dt.strftime("%d/%m/%Y")
df_cotas = df_cotas.replace([np.nan, np.inf, -np.inf], '')
df_cotas

: 

In [ ]:
pagina.update(
    range_name='a2', 
    values=[df_cotas.columns.values.tolist()] + df_cotas.values.tolist()
)

: 